In [1]:
import hashlib
from py_ecc.secp256k1 import *
import sha3
import ecdsa
import os
from eth_account import Account
from typing import NewType
from functools import wraps
from dataclasses import dataclass

SECP256K1_ORDER = int(0xFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFEBAAEDCE6AF48A03BBFD25E8CD0364141)

USE_VIEWTAGS = True
LOGGING = False

In [2]:
def print_function_name(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        if not LOGGING:
            return func(*args, **kwargs)
        print(f"Executing function: {func.__name__}")
        return func(*args, **kwargs)
    return wrapper
pfn = print_function_name

In [99]:
PrivateKey = NewType('PrivateKey', int)
PublicKeyPoint = NewType('PublicKeyPoint', int)
HashBelowOrder = NewType('HashBelowOrder', int)
EthAddress = NewType('EthAddress', str)
ViewTag = NewType('ViewTag', int)
HexString = NewType('HexString', str)
CompressedPublicKey = NewType('CompressedPublicKey', str)
StealthMetaAddress = NewType('StealthMetaAddress', str)



@dataclass
class PublicKey:
    x: PublicKeyPoint = 0
    y: PublicKeyPoint = 0

@pfn
def generate_random_private_key() -> PrivateKey:
    # Generate a random 32 bytes (256 bits)
    random_bytes = os.urandom(32)

    # Convert the random bytes to an integer
    random_int = int.from_bytes(random_bytes, 'big')

    # Modulo it by the order of the curve to ensure it's a valid private key
    private_key = random_int % SECP256K1_ORDER

    # Ensure the private key is not 0
    if private_key == 0:
        return generate_random_private_key()

    return PrivateKey(private_key)

@pfn
def to_hex(i: int) -> HexString:
    return hex(privateKey)

@pfn
def priv_to_pub(privateKey: PrivateKey) -> PublicKey:
    # Convert the private key integer to a 32-byte big-endian representation and get the public key
    return PublicKey(*secp256k1.privtopub(privateKey.to_bytes(32, 'big')))

@pfn
def pubkey_add(publicKey: PublicKey, publicKey2: PublicKey):
    return PublicKey(
        *secp256k1.add(
            (publicKey.x, publicKey.y),
            (publicKey2.x, publicKey2.y)
        )
    )
    
@pfn
def pubkey_mul(publicKey: PublicKey, x: int) -> PublicKey:
    return PublicKey(*secp256k1.multiply((publicKey.x, publicKey.y), x))

@pfn
def hash_public_key(publicKey: PublicKey) -> HashBelowOrder:
    return HashBelowOrder(
        int(
            sha3.keccak_256(
                publicKey.x.to_bytes(32, "big") + 
                publicKey.y.to_bytes(32, "big")
            ).hexdigest(), 
            16
        ) % SECP256K1_ORDER
    )

@pfn
def pubkey_to_address(publicKey: PublicKey) -> EthAddress:
    return EthAddress(
        "0x" + sha3.keccak_256(
            publicKey.x.to_bytes(32, "big") +
            publicKey.y.to_bytes(32, "big")
        ).hexdigest()[-40:]
    )

@pfn
def uncompress_public_key(compressedPublicKey: CompressedPublicKey) -> PublicKey:
    key = ecdsa.VerifyingKey.from_string(compressedKeyBytes, curve=ecdsa.SECP256k1)
    publicKey = uncompress_public_key(compressedPublicKey)
    return PublicKey(publicKey.x(), publicKey.y())

@pfn
def parse_stealth_meta_address(stealthMetaAddress: StealthMetaAddress) -> (PublicKey):
    if not stealthMetaAddress.startswith("st:eth:0x"):
        return
    if not len(stealthMetaAddress) == 66*2+2+7:
        return
    
    sma_string = input_str.split(":")[-1][2:] # This gets the part after 'st:eth:0x'
    publicKeySp, publicKeySc = sma_string[:66], sma_string[66:] #
    return uncompress_public_key(publicKeySp), uncompress_public_key(publicKeySc)


@pfn
def compute_ephemeral_public_key(ephemeralPrivateKey: PrivateKey) -> PublicKey:
    return priv_to_pub(ephemeralPrivateKey)

@pfn
def compute_dh_secret(publicKey: PublicKey, ephemeralPrivateKey: PrivateKey) -> PublicKey:
    return pubkey_mul(publicKey, ephemeralPrivateKey)
    
@pfn
def hashed_dh_secret_to_point(dhSecretHash: HashBelowOrder) -> PublicKey:
    return priv_to_pub(dhSecretHash)

@pfn
def hash_dh_secret(dhSecret: PublicKey) -> HashBelowOrder:
    return hash_public_key(dhSecret)

@pfn
def retrieve_view_tag(dhSecretHash: HashBelowOrder) -> ViewTag:
    return ViewTag(dhSecretHash.to_bytes(32, "big")[0])

@pfn
def compute_stealth_public_key_from_dh_point(dhSecretPoint: PublicKey, publicKey: PublicKey) -> PublicKey:
    return pubkey_add(publicKey, dhSecretPoint)

@pfn
def check_view_tag(dhSecretHash: HashBelowOrder, comparisonTag: ViewTag):
    retrievedViewTag = retrieve_view_tag(dhSecretHash)
    return True if retrievedViewTag == comparisonTag else False
 

@pfn
def compute_stealth_address_from_recipient_info(
    publicKeySpending: PublicKey, 
    publicKeyScanning: PublicKey, 
    ephemeralPrivateKey: PrivateKey
) -> (EthAddress, PublicKey, ViewTag):
    dhSecret = compute_dh_secret(publicKeyScanning, ephemeralPrivateKey)
    dhSecretHash = hash_dh_secret(dhSecret)
    viewTag = retrieve_view_tag(dhSecretHash)  
    dhSecretPoint = hashed_dh_secret_to_point(dhSecretHash)
    stealthAddressPublicKey = compute_stealth_public_key_from_dh_point(dhSecretPoint, publicKeySpending)
    stealthAddress = pubkey_to_address(stealthAddressPublicKey)
    ephemeralPublicKey = compute_ephemeral_public_key(ephemeralPrivateKey)
    return stealthAddress, ephemeralPublicKey, viewTag


@pfn
def generate_stealth_address_info_from_meta_address(
    stealthMetaAddress: StealthMetaAddress, 
    ephemeralPrivateKey: PrivateKey
) -> (EthAddress, PublicKey, ViewTag):
    publicKeySpending, publicKeyScanning = parse_stealth_meta_address(stealthMetaAddress)
    return compute_stealth_address_from_recipient_info(publicKeySpending, publicKeyScanning, ephemeralPrivateKey)
    

@pfn
def parse(
    publicKeys: [PublicKey], 
    stealthAddresses: [EthAddress],
    viewTags: [ViewTag],
    scanningPrivateKey: PrivateKey, 
    spendingPublicKey: PublicKey,
) -> ([(HashBelowOrder, EthAddress)], [EthAddress]):
    found = []
    parsed = []
    for pubkey, stA, viewtag in zip(publicKeys, stealthAddresses, viewTags):
        dhSecret = compute_dh_secret(pubkey, scanningPrivateKey)
        dhSecretHash = hash_dh_secret(dhSecret)
        if not check_view_tag(dhSecretHash, viewtag):
            parsed.append(stA)
            continue
        dhSecretPoint = hashed_dh_secret_to_point(dhSecretHash)
        stealthAddressPublicKey = compute_stealth_public_key_from_dh_point(dhSecretPoint, spendingPublicKey)
        stealthAddress = pubkey_to_address(stealthAddressPublicKey)
        if stealthAddress == stA:
            print(f"Stealth address found | {stA}")
            found.append((dhSecretHash, stA))
        parsed.append(stA)
    return found, parsed


@pfn
def compute_stealth_address_private_key(
    dhSecrets: [HashBelowOrder], 
    spendingPrivateKey: PrivateKey
) -> [PrivateKey]:
    return [(spendingPrivateKey + dh_secret) % SECP256K1_ORDER for dh_secret in dhSecrets]
        

In [100]:
aPriv = generate_random_private_key()
aPub = priv_to_pub(aPriv)

In [101]:
bPrivSC = generate_random_private_key()
bPrivSP = generate_random_private_key()
bPubSP = priv_to_pub(bPrivSP)
bPubSC = priv_to_pub(bPrivSC)


In [102]:
bPubSC

PublicKey(x=62603816693165357161519198575516634423088782743611591198174083658109481580263, y=114504888317688553261153475312333677499185822781284220130304107510717073626768)

In [103]:
stA, ephemeralPublicKey, viewTag = compute_stealth_address_from_recipient_info(bPubSP, bPubSC, aPriv)

In [104]:
viewTag

37

In [105]:
found, _ = parse([ephemeralPublicKey], [stA], [viewTag], bPrivSC, bPubSP)

Stealth address found | 0x75255c7fe303cde8681ba61355b02fc0cf0da9cc


In [110]:
found

[(17151583196944826877136772214029804164759284870419242749617404751329001376863,
  '0x75255c7fe303cde8681ba61355b02fc0cf0da9cc')]

In [106]:
dh_secrets = [i[0] for i in found]
compute_stealth_address_private_key(dh_secrets,bPrivSP)

[45933781020337241433236171727533356179575802717288149974187345313975658392259]

In [107]:
hex(45933781020337241433236171727533356179575802717288149974187345313975658392259)

'0x658d9946d9820792cda0034ee32f1313ba784f3090df24beefcb72f876e4aac3'

In [109]:
pubkey_to_address(priv_to_pub(45933781020337241433236171727533356179575802717288149974187345313975658392259))

'0x75255c7fe303cde8681ba61355b02fc0cf0da9cc'